Дз №2:
Необходимо предсказать риск проблем с сердцем (колонка output) на основе данных (необходима регистрация на kaggle).  Можно использовать материалы ноутбука занятия и наработки сообщества (вкладка Code) - но не бездумно копировать! Задание на отлично: рассмотреть не только knn, но и другие классификаторы, попробовать обучить несколько из них и сравнить полученный результат по метрикам качества на тесте.

Критерии, по которым проверяется работа:
Выполнена ли загрузка данных? (корректно ли загружен файл, все ли загрузилось и в правильном ли формате)
Есть ли первичное исследование? (аномалии, пропуски, корректность заполнения)
Есть ли основное исследование (EDA)? (баланс классов, линейные корреляции, группировки с целевым, тезисные выводы словами)
Есть ли предобработка? (кодирование данных - one-hot/label encoder, нормализация, train/val/test или train/test split, стратификация по балансировке)
Есть ли обучение алгоритма и перебор гиперпараметров? (grid search и не только, обучение алгоритма c лучшими параметрами)
Есть ли оценка алгоритма? (Разные метрики качества? Интерпретация ошибок)

Опциональные критерии на отлично (8-10 /10):
Есть перебор параметров отличных от knn алгоритмов?
Есть ли аугментация признаков?
Есть ли аугментация данных на train части?
Есть ли сравнение разных вариантов кодирования данных?
Есть ли обучение отличных (тык) от sklearn алгоритмов? (введение в pytorch хороший официальный туториал и туториалы на русском 1 и 2)
Выполнение на 10 не требует всех пунктов выше, но чем больше - тем вероятнее.

И самый важный критерий оценивания этого дз и любых дальнейших:
Загрузить в ноутбуке занятия вместо .csv файла занятия  указанный в задании, просто запустив все ячейки, конечно, можно. Но это не добавит вам ни понимания ни оценки. Важно продемонстрировать свою способность  выходить за рамки показанного в занятии (хоть минимально), исследовать документацию pandas, sklearn или seaborn, пробовать новые техники визуализации, подготовки и обучения (поначалу копировать, с пониманием, подходы сообщества - вкладка code можно). Ваши текстовые тезисные комментарии показывающие вашу логику в обработке от и до крайне важны. Какой-то скопированный код может не запуститься, но если видно, что были предприняты попытки разобраться и краткий комментарий что пошло не так - это большой плюс в ваш опыт и оценку. БОльшая часть работы специалиста по данным - отладка, исследование документаций библиотек и проверка своих гипотез, перебор разных параметров, без этого опыта построить что-либо выходящее за рамки .fit_predict() из sklearn не получится.


загрузка

In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import figure


очистка

In [50]:
df = pd.read_csv('./data/heart.csv')
print(len(df))
df = df.drop_duplicates()

# df = df.drop(labels=df[df['age'] > 100])
print(len(df))
# df['age'].value_counts()
index_drop = df[df['age'] > 100].index
df = df.drop(index_drop)
print(len(df))
# df['age'].value_counts()

# figure(figsize = (10, 15))
# sns.heatmap(df == ' ?', cbar=False)

# df.describe()

# cols = df.columns[:30] # первые 30 колонок
# определяем цвета 
# желтый - пропущенные данные, синий - не пропущенные
# colours = ['#000099', '#ffff00'] 
# sns.heatmap(df[cols].isnull(), cmap=sns.color_palette(colours))

df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)

# df['age'].hist(bins=100)
# df.boxplot(column=['age'])
# df['age'].describe()
# df['cp'].value_counts().plot.bar()
num_rows = len(df.index)
low_information_cols = [] #

# for col in df.columns:
#     cnts = df[col].value_counts(dropna=False)
#     top_pct = (cnts/num_rows).iloc[0]
    
#     if top_pct > 0.7:
#         low_information_cols.append(col)
#         print('{0}: {1:.5f}%'.format(col, top_pct*100))
#         print(cnts)
#         print()

print(df.shape)


307
305
303
['age' 'sex' 'cp' 'trtbps' 'chol' 'fbs' 'restecg' 'thalachh' 'exng'
 'oldpeak' 'slp' 'caa' 'thall' 'output']
(303, 14)


In [55]:
from nltk.metrics import edit_distance

df_city_ex = pd.DataFrame(data={'city': ['torontoo', 'toronto', 'tronto', 'vancouver', 'vancover', 'vancouvr', 'montreal', 'calgary']})


df_city_ex['city_distance_toronto'] = df_city_ex['city'].map(lambda x: edit_distance(x, 'toronto'))
df_city_ex['city_distance_vancouver'] = df_city_ex['city'].map(lambda x: edit_distance(x, 'vancouver'))
# df_city_ex

msk = df_city_ex['city_distance_toronto'] <= 2
# print(msk)
df_city_ex.loc[msk, 'city'] = 'toronto'

msk = df_city_ex['city_distance_vancouver'] <= 2
df_city_ex.loc[msk, 'city'] = 'vancouver'

df_city_ex

,city,city_distance_toronto,city_distance_vancouver
0,toronto,1,8
1,toronto,0,8
2,toronto,1,8
3,vancouver,8,0
4,vancouver,7,1
5,vancouver,7,1
6,montreal,7,8
7,calgary,7,8
